<a href="https://colab.research.google.com/github/MuhammadBasilKhan/Fine-tune-model/blob/main/vgg16_on_mnistdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
pip install tensorflow


In [3]:
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow.keras as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

In [4]:
vgg16_model = VGG16(weights='imagenet',
                    include_top=False,
                    classes=10,
                    input_shape=(32,32,3)# input: 32x32 images with 3 channels -> (32, 32, 3) tensors.
                   )

58889256/58889256 [==============================] - 1s 0us/step


In [5]:
model = Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

In [6]:
from tensorflow.keras.layers import Dense, Flatten, Dropout
model.add(Flatten())
model.add(Dense(512, activation='relu', name='hidden1'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu', name='hidden2'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax', name='predictions'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0         
                                                                 
 block3_conv1 (Conv2D)       (None, 8, 8, 256)         2

In [7]:
(X_train, y_train), (X_test, y_test) = k.datasets.cifar10.load_data()

print("******************")
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

170498071/170498071 [==============================] - 5s 0us/step
******************
(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [8]:
y_train_ohe = to_categorical(y_train, num_classes = 10)
y_test_ohe = to_categorical(y_test, num_classes = 10)

In [9]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train  /= 255
X_test /= 255

print("******************")
print(X_train.shape)
print(y_train_ohe.shape)
print(X_test.shape)
print(y_test_ohe.shape)

******************
(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


In [10]:
X_val = X_train[40000:]
y_val = y_train_ohe[40000:]
print(X_val.shape)
print(y_val.shape)

(10000, 32, 32, 3)
(10000, 10)


In [11]:
X_train = X_train[:40000]
y_train_ohe = y_train_ohe[:40000]
print(X_train.shape)
print(y_train_ohe.shape)

(40000, 32, 32, 3)
(40000, 10)


In [12]:
sgd = optimizers.SGD(lr=0.001, momentum=0.9)

# For a multi-class classification problem
model.compile(loss='categorical_crossentropy',optimizer= sgd,metrics=['accuracy'])


def lr_scheduler(epoch):
    return 0.001 * (0.5 ** (epoch // 20))
reduce_lr = LearningRateScheduler(lr_scheduler)

mc = ModelCheckpoint('./weights.h5', monitor='val_accuracy', save_best_only=True, mode='max')


# initialize the number of epochs and batch size
EPOCHS = 100
BS = 128

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# train the model
history = model.fit_generator(
    aug.flow(X_train,y_train_ohe, batch_size=BS),
    validation_data=(X_val,y_val),
    steps_per_epoch=len(X_train) // BS,
    epochs=EPOCHS,
    callbacks=[reduce_lr,mc])

#We load the best weights saved by the ModelCheckpoint
model.load_weights('./weights.h5')

Epoch 1/100


<ipython-input-12-87337afcb376>:29: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


312/312 [==============================] - ETA: 0s - loss: 1.7449 - accuracy: 0.3686

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


312/312 [==============================] - 44s 111ms/step - loss: 1.7449 - accuracy: 0.3686 - val_loss: 1.2071 - val_accuracy: 0.5694 - lr: 0.0010
Epoch 2/100
312/312 [==============================] - 30s 95ms/step - loss: 1.1766 - accuracy: 0.5904 - val_loss: 0.8505 - val_accuracy: 0.7002 - lr: 0.0010
Epoch 3/100
312/312 [==============================] - 30s 96ms/step - loss: 1.0003 - accuracy: 0.6603 - val_loss: 0.7392 - val_accuracy: 0.7473 - lr: 0.0010
Epoch 4/100
312/312 [==============================] - 30s 96ms/step - loss: 0.8992 - accuracy: 0.6962 - val_loss: 0.6945 - val_accuracy: 0.7597 - lr: 0.0010
Epoch 5/100
312/312 [==============================] - 29s 94ms/step - loss: 0.8324 - accuracy: 0.7207 - val_loss: 0.6076 - val_accuracy: 0.7893 - lr: 0.0010
Epoch 6/100
312/312 [==============================] - 30s 95ms/step - loss: 0.7790 - accuracy: 0.7398 - val_loss: 0.5637 - val_accuracy: 0.8087 - lr: 0.0010
Epoch 7/100
312/312 [==============================] - 29s 94ms

In [13]:
train_loss, train_accuracy = model.evaluate_generator(aug.flow(X_train,y_train_ohe, batch_size=BS), 156)
print('Training loss: {}\nTraining accuracy: {}'.format(train_loss, train_accuracy))

<ipython-input-13-a5a972341de3>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  train_loss, train_accuracy = model.evaluate_generator(aug.flow(X_train,y_train_ohe, batch_size=BS), 156)


Training loss: 0.17255964875221252
Training accuracy: 0.9396033883094788


In [14]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print('Validation loss: {}\nValidation accuracy: {}'.format(val_loss, val_accuracy))

313/313 [==============================] - 3s 8ms/step - loss: 0.3814 - accuracy: 0.8869
Validation loss: 0.3813720941543579
Validation accuracy: 0.886900007724762


In [15]:
test_loss, test_accuracy = model.evaluate(X_test,y_test_ohe,)
print('Testing loss: {}\nTesting accuracy: {}'.format(test_loss, test_accuracy))

313/313 [==============================] - 3s 8ms/step - loss: 0.4074 - accuracy: 0.8834
Testing loss: 0.40743446350097656
Testing accuracy: 0.883400022983551
